In [1]:
%load_ext autoreload   
%autoreload 2

from src import *
import dill
import nltk
import os
nltk.download('all')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\piotr\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\piotr\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\piotr\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\piotr\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\piotr\AppData\Roaming\nltk_data...
[

True

In [2]:
dump_filename = 'res/sessions/daigt_dataset.db'
writing_style_analysis_dump_filename = 'res/sessions/writing_style_dataset.db'

dump_dict = None
with open(writing_style_analysis_dump_filename, 'rb') as file:
    dump_dict = dill.load(file)

writing_style_feature_extractor: FeatureExtractor = dump_dict['feature_extractor']
writing_style_pca_analysis_results: FeatureExtractor = dump_dict['pca_analysis_results']

In [3]:
settings = Settings()

In [4]:
dataset = DaigtDataset(settings)
dataset.load()

In [5]:
dataset.clean()

In [6]:
print(f"Text removed while cleaning: {DaigtMetadataAnalysis.get_percentage_of_removed_text(dataset)}%.")

Text removed while cleaning: 0.012379704917830402%.


In [7]:
preprocessing_results = DaigtPreprocessing(settings).preprocess(dataset)
preprocessing_results.info()

,collection,num_of_syllabes,num_of_complex_words,num_of_sentences,num_of_words,num_of_splits
0,human,16534054,1132449,571895,11498133,12626085
1,mistral,10679300,1303559,285237,6069118,6776800
2,llama,8925648,901248,295540,5576315,6250480
3,gpt,2102310,245189,61041,1220047,1368602
4,claude,1130777,140499,34997,621079,686557
5,falcon,3025621,377694,84050,1708179,1909399
6,palm,1081158,109939,42875,678377,771497
7,cohere,182736,19642,5791,112957,125297
8,ada,236731,21785,8902,154673,170596
9,babbage,212813,20728,7751,136981,150379


In [8]:
metrics_analysis_results = DaigtMetricsAnalysis().analyze(preprocessing_results)

In [9]:
writing_style_feature_extractor.get_feature_names()

['source_name',
 'author_name',
 'collection_name',
 'unique_word_count',
 'average_word_length',
 'average_sentence_length',
 'average_syllables_per_word',
 'flesch_reading_ease',
 'flesch_kincaid_grade_level',
 'gunning_fog_index',
 'yules_characteristic_k',
 'herdans_c',
 'maas',
 'simpsons_index',
 ';',
 '"',
 "'",
 '*',
 '-',
 ',',
 '.',
 'it',
 'my',
 'its',
 'was',
 'as',
 'she',
 'with',
 'that',
 'and',
 'a',
 'is',
 'you',
 'our',
 'his',
 'of',
 'he',
 'in',
 'their',
 'said',
 'the',
 'to',
 'her']

In [10]:
pca_analysis = DaigtPCAAnalysis(
    settings=settings,
    writing_style_feature_extractor=writing_style_feature_extractor,
    writing_style_pca_analysis_results=writing_style_pca_analysis_results
)
pca_results = pca_analysis.analyze(metrics_analysis_results)

In [11]:
DaigtPCAVisualization(pca_results).visualize()

In [ ]:
if os.path.exists(dump_filename):
    os.remove(dump_filename)
with open(dump_filename, 'wb') as file:
    dill.dump(
        obj={
            "dataset": dataset,
            "preprocessing_results": preprocessing_results,
            "metrics_analysis_results": metrics_analysis_results,
            "feature_extractor": writing_style_feature_extractor,
            "pca_results": pca_results,
        },  
        file=file
    )